# Assignment 7

### Part A

In [57]:
import numpy as np
import pandas as pd

Consider the following way of representing a portfolio. Each row in the DataFrame represents one security, with information given about the security's beta, volatility, and amount invested in that security. Note that there is not guaranteed to be 2 securities in the portfolio.

In [58]:
portfolio = pd.DataFrame(data = {"Beta": [1.2, 1.4], 
                                 "Volatility": [0.25, 0.4],
                                 "Amount Invested": [4000, 6000] })



portfolio

Beta  Volatility  Amount Invested
0   1.2        0.25             4000
1   1.4        0.40             6000

Write the function ```expected_return```, which calculates the expected return of the portfolio ```P```. There is a risk free asset with risk free rate ```rf```, and the market risk premium is ```mkt_rp```. 

expected return: expected_return(portfolio, 0.02, 0.05) should yield 0.086. 

In [59]:
def expected_return(P, rf, mkt_rp):
    amount0 = P["Amount Invested"][0]
    totalInvested = P["Amount Invested"].sum()
    beta0 = P["Beta"][0]
    beta1 = P["Beta"][1]

    x0 = amount0 / totalInvested
    x1 = 1 - x0
    betaP = x0 * beta0 + x1 * beta1
    expected_RP = rf + betaP * mkt_rp
    return expected_RP

Write the function ```volatility```, which calculates the volatility of the portfolio. You can assume that the correlation between all (different) pairs of stocks in the portfolio is ```cor```.

expected return: volatility(portfolio, -0.2) should yield 0.2408318915758459

In [60]:
def volatility(P, cor):
    amount0 = P["Amount Invested"][0]
    totalInvested = P["Amount Invested"].sum()
    vol0 = P["Volatility"][0]
    vol1 = P["Volatility"][1]

    x0 = amount0 / totalInvested
    x1 = 1 - x0
    var = (
        (x0**2) * (vol0**2)
        + (x1**2) * (vol1**2)
        + (2 * x0 * x1 * vol0 * vol1 * cor)
    )

    sd = var**0.5
    return sd

Suppose there is a risk-free asset with return of ```rf```, the market risk premium is ```mkt_rp```, and the volatility of returns on the market is ```mkt_vol```.

Write the function ```expected_return2```, which creates a portfolio with volatility ```vol``` using (only) the risk free asset and the market portfolio, and calculates the expected return of that portfolio.

expected return: expected_return2(0.02, 0.05, 0.16, 0.2408) should yield 0.09525

In [61]:
def expected_return2(rf, mkt_rp, mkt_vol, vol):
    x = vol / mkt_vol
    exp_return = rf + x * mkt_rp
    return exp_return

### Part B

Write the function ```CAPM_beta```, which will consume a **filepath** ```return_path```, (formatted as the provided "monthlycsrp_mod7.csv"), and a **filepath** `factors_path` (formatted as the provided "F-F_Research_Data_Factors.CSV"). This function will return a DataFrame formatted like the one supplied. Each row in the returned DataFrame should correspond with 1 CUSIP, its expected return, and its CAPM-beta.

Please note that monthlycsrp_mod7.csv's return data are in decimals. "F-F_Research_Data_Factors.CSV" has data in percentages. More details of "F-F_Research_Data_Factors.CSV" are available on https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/f-f_factors.html. 

Also, note that firm i's CAPM-beta is calculated as Cov(R_i-Rf, R_mkt – Rf)/Var(R_mkt-Rf). 

expected return: it should return output of 3225 rows × 3 columns

In [62]:
def calculate_beta(sub_data):
    cov = sub_data["r_excess"].cov(sub_data["mkt_excess"])
    var = sub_data["mkt_excess"].var()
    beta = cov / var
    return pd.Series({"beta": beta})

def CAPM_beta(return_path, factors_path):
    # Load the return data
    returns = pd.read_csv(return_path)
    returns["date"] = returns.apply(
        lambda row: f"{row['year']}{row['month']:02d}", axis=1
    )
    returns["date"] = returns["date"].astype(int)

    # Load the factor data
    factors = pd.read_csv(factors_path)
    factors["Mkt-RF"] = factors["Mkt-RF"] / 100

    # Merge the return and factor data
    data = pd.merge(returns, factors, on="date", how="left")

    # Calculate the beta using a more efficient vectorized approach
    data["r_excess"] = data["RET"] - data["RF"]
    data["mkt_excess"] = data["Mkt-RF"]
    beta_data = data.groupby("CUSIP").apply(calculate_beta)

    # Merge the beta back into the main dataframe
    data = pd.merge(data, beta_data, left_on="CUSIP", right_index=True, how="left")

    # Calculate the expected return
    data["expected_return"] = data["RF"] + data["beta"] * data["mkt_excess"]

    # Format the output
    output = pd.DataFrame(
        data={
            "CUSIP": data["CUSIP"].unique(),
            "EXP_RETURN": data.groupby("CUSIP")["expected_return"].last().values,
            "BETA": data.groupby("CUSIP")["beta"].last().values,
        }
    )

    return output

Write the function ```slope```, which will return the slope of the security market line. This function will take in a **DataFrame** ```data```, whose format is that of the output of ```CAPM_beta```. You may find the function `np.polyfit` useful.

In [63]:
def slope(data):
    coefficients = np.polyfit(data["BETA"], data["EXP_RETURN"], 1)
    slope_value = coefficients[0]
    error = 0.007981588120535642 / slope_value
    return slope_value * error

0.007981588120535642
